<a href="https://colab.research.google.com/github/PedroTechy/DataProcessingEdit/blob/main/spark_streaming/examples/example_3_api_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [1]:
%pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').getOrCreate()
sc = spark.sparkContext

In [3]:
!rm -rf /content/landing
!rm -rf /content/bronze
!mkdir -p /content/landing

## Simulate producer:
- extract data from API
- store data as json in the lake
- run task async

In [4]:
import requests
from pyspark.sql.types import *
import json
import datetime
import asyncio


# this setup is kind to simulate streaming using an api

async def ingest_from_api(url: str, table: str, schema: StructType = None):
  response = requests.get(url)
  timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
  if response.status_code == 200:
    data = response.json()
    with open(f"/content/landing/{table}_{int(timestamp)}.json", "w") as f:
        json.dump(data, f)

async def producer(loop: int, interval_time: int):
  for i in range(loop):
    await ingest_from_api("https://api.carrismetropolitana.pt/vehicles", "vehicles")
    await ingest_from_api("https://api.carrismetropolitana.pt/lines", "lines")
    await asyncio.sleep(interval_time)

async def main():
  asyncio.create_task(producer(10, 30))

await main()

- Read from /content/landing as streaming
- store data in memory (for testing)
- store data in the bronze layer

In [5]:
from pyspark.sql.types import *

vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])

stream = spark.readStream.format("json").schema(vehicle_schema).load("/content/landing/vehicles*")

dedup = stream.dropDuplicates()

In [6]:
# using memory for testing

try:
  if query.isActive:
    query.stop()
except:
  pass

query = (dedup.writeStream.format("memory").option("queryName", "vehicles").start())

In [10]:
query.recentProgress

[{'id': 'c6b620f1-f2c8-44d5-b34d-2661dbedf2e1',
  'runId': 'cd190fe9-5479-4ffe-a708-ab7d91971e36',
  'name': 'vehicles',
  'timestamp': '2024-11-30T09:55:34.757Z',
  'batchId': 0,
  'numInputRows': 4240,
  'inputRowsPerSecond': 0.0,
  'processedRowsPerSecond': 156.50954191428886,
  'durationMs': {'addBatch': 25306,
   'commitOffsets': 44,
   'getBatch': 310,
   'latestOffset': 195,
   'queryPlanning': 984,
   'triggerExecution': 27086,
   'walCommit': 196},
  'stateOperators': [{'operatorName': 'dedupe',
    'numRowsTotal': 3785,
    'numRowsUpdated': 3785,
    'allUpdatesTimeMs': 8591,
    'numRowsRemoved': 0,
    'allRemovalsTimeMs': 7,
    'commitTimeMs': 7167,
    'memoryUsedBytes': 1655184,
    'numRowsDroppedByWatermark': 0,
    'numShufflePartitions': 200,
    'numStateStoreInstances': 200,
    'customMetrics': {'loadedMapCacheHitCount': 0,
     'loadedMapCacheMissCount': 0,
     'numDroppedDuplicateRows': 455,
     'stateOnCurrentVersionSizeBytes': 1626384}}],
  'sources': [{'d

In [9]:
#query.lastProgress -> contains information on the last streaming progress, good for troubleshooting
#query.recentProgress -> same as last, but with more recent events and not just the last one

spark.sql("select * from vehicles").show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|    100|             4708-21| IN_TRANSIT_TO| 42|2507| 38.88752|   2764|-9.125865|  2764_0_2|  2764_0|            SCHEDULED|        4708|5.8333335| 071149|2024-11-30 09:45:26|2764_0_2|2|1|0900...|
|    322|             4508-21| IN_TRANSIT_TO| 42|2565|38.807926|   2523|-9.203173|  2523_0_1|  2523_0|            SCHEDULED|        4508|11.111111| 110643|2024-11-30 09:45:33|2523_0_1|2|1|0925...|
|     71|202411

In [11]:
!rm -rf /content/bronze

In [12]:
query.stop()

In [14]:
from pyspark.sql.functions import *

# watermark is necessary because of the aggregation (here i am also using the stream data frame that i defiend early)
transformed = stream.withWatermark("timestamp", "60 seconds")
agg = (transformed
       .groupBy(window(transformed.timestamp, "5 minutes"), col("current_status"))
       .agg(min(transformed.timestamp).alias("init_timestamp"), count("*").alias("count")))

def insert_vehicles(df, batch_id):
  #df2 = df.groupBy("window").pivot("current_status").sum("count")
  df.write.format("parquet").mode("append").save("/content/bronze/vehicles")

# using memory for testing
query2 = (agg
          .writeStream # the write stream need the foreachBatch because i am using the parquet to save
          .outputMode("append")
          .foreachBatch(insert_vehicles)
          .option("checkpointLocation", "/content/bronze/checkpoint")
          .trigger(processingTime='20 seconds')
          .start())

In [21]:
spark.read.format("parquet").load("/content/bronze/vehicles/*").show(100, False)

+------------------------------------------+--------------+-------------------+-----+
|window                                    |current_status|init_timestamp     |count|
+------------------------------------------+--------------+-------------------+-----+
|{2024-11-30 09:40:00, 2024-11-30 09:45:00}|IN_TRANSIT_TO |2024-11-30 09:41:49|1060 |
|{2024-11-30 09:40:00, 2024-11-30 09:45:00}|INCOMING_AT   |2024-11-30 09:41:49|370  |
|{2024-11-30 09:40:00, 2024-11-30 09:45:00}|STOPPED_AT    |2024-11-30 09:41:52|510  |
+------------------------------------------+--------------+-------------------+-----+



## Report
- show vehicles by status in 5-min window time
- one line per window time

In [22]:
def pivot_data(df: DataFrame):
  result = df.orderBy("init_timestamp").groupBy("window").pivot("current_status").sum("count")
  result.show(100, False)

df = spark.read.format("parquet").load("/content/bronze/vehicles/*")
pivot_data(df)

+------------------------------------------+-----------+-------------+----------+
|window                                    |INCOMING_AT|IN_TRANSIT_TO|STOPPED_AT|
+------------------------------------------+-----------+-------------+----------+
|{2024-11-30 09:40:00, 2024-11-30 09:45:00}|370        |1060         |510       |
+------------------------------------------+-----------+-------------+----------+



In [ ]:
query2.stop()

ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "<ipython-input-48-9446b2336b0e>", line 11, in insert_vehicles
    df.write.format("parquet").mode("append").save("/content/bronze/vehicles")
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/readwriter.py", line 1463, in save
    self._jwrite.save(path)
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.10/dist-packages/pysp